# Paper tutorial

In [ ]:
import pybinding as pb
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
import shutil
import subprocess
import kite
from matplotlib import gridspec

pb.pltutils.use_style()

## Square lattice

In [ ]:
def square_lattice(onsite=(0, 0)):
    """Make a square lattice with nearest neighbor hopping"""

    a1 = np.array([1, 0])
    a2 = np.array([0, 1])
    # create a lattice with 2 primitive vectors
    lat = pb.Lattice(
        a1=a1, a2=a2
    )
    # Add sublattices
    lat.add_sublattices(
        # name, position, and onsite potential
        ('A', [0, 0], onsite[0])
    )
    # Add hoppings
    lat.add_hoppings(
        ([1, 0], 'A', 'A', - 1),
        ([0, 1], 'A', 'A', - 1)
    )
    return lat

In [ ]:
lattice = square_lattice()
# number of decomposition parts
n1 = n2 = 1
# number of unit cells in each direction.
l1 = l2 = 512
# number of polynomials
num_moments = 256
configuration = kite.Configuration(divisions=[n1, n2], length=[l1, l2], boundaries=[True, True],
                                   is_complex=False, precision=1)
# require the calculation of singleshot_conductivity_dc
calculation = kite.Calculation(configuration)
direction = 'xx'
calculation.dos(num_points=1000, num_moments=256, num_random=10, num_disorder=1)
calculation.conductivity_dc(num_points=1000, num_moments=num_moments, num_random=1, num_disorder=1,
                            direction=direction, temperature=1)
# configure the *.h5 file
filename = 'cond_dos_square_lat.h5'
kite.config_system(lattice, configuration, calculation, filename=filename)

In [ ]:
pb.utils.tic()  # measure the time
kite.KITEx(filename)
pb.utils.toc('Time for moments calculation ')

pb.utils.tic()  # measure the time
kite.KITEtools(filename + ' --CondDC -T 300 -F -4 4 10000 -E 4000 -K green 0.01 -N condDC_{}.dat'.format(direction))
kite.KITEtools(filename + ' --DOS -F -3 3 1000 -K green 0.01 -N dos.dat')
pb.utils.toc('Time for reconstructing ')

In [ ]:
dos_raw = np.loadtxt('dos.dat')
cond_raw = np.loadtxt('condDC_{}.dat'.format(direction))

plt.figure()
ax1 = plt.gca()
ax1.plot(cond_raw[:,0], cond_raw[:,1])
ax1.set_xlabel(r'$\mathrm{E~(eV)}$')
ax1.set_ylabel(r'$\mathrm{\sigma_{xx}~(e^2/h)}$')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(dos_raw[:,0], dos_raw[:,1], c='C1')
plt.sca(ax2)
ax2.tick_params(axis='y', labelcolor='C1')
ax2.set_ylabel(r'$\mathrm{DOS~(a.u.)}$')

## Graphene in magnetic field

In [ ]:
from pybinding.repository import graphene
from math import sqrt
a = 0.24595  # [nm] unit cell length
a_cc = 0.142 # [nm] carbon-carbon distance
t = -2.8     # [eV] nearest neighbour pz-pz hopping

def monolayer_4atom(onsite=(0, 0)):
    """Nearest-neighbor with 4 atoms per unit cell: square lattice instead of oblique

    Parameters
    ----------
    onsite : Tuple[float, float]
        Onsite energy for sublattices A and B.
    """

    lat = pb.Lattice(a1=[a, 0], a2=[0, 3*a_cc])

    lat.add_sublattices(('A',  [  0, -a_cc/2], onsite[0]),
                        ('B',  [  0,  a_cc/2], onsite[1]),
                        ('C', [a / 2, a_cc], onsite[0]),
                        ('D', [a / 2, 2 * a_cc], onsite[1]))

    lat.add_hoppings(
        # inside the unit sell
        ([0, 0], 'A',  'B',  t),
        ([0, 0], 'B',  'C', t),
        ([0, 0], 'C', 'D', t),
        # between neighbouring unit cells
        ([-1, -1], 'A', 'D', t),
        ([ 0, -1], 'A', 'D', t),
        ([-1,  0], 'B', 'C', t),
    )

    lat.min_neighbors = 2
    return lat

lat = monolayer_4atom()
bfield = 154.19
modification = kite.Modification(magnetic_field=bfield)
disorder = kite.Disorder(lat)
dis_std = 0.4
disorder.add_disorder('A', 'Uniform', +0.0, dis_std)
disorder.add_disorder('B', 'Uniform', +0.0, dis_std)
disorder.add_disorder('C', 'Uniform', +0.0, dis_std)
disorder.add_disorder('D', 'Uniform', +0.0, dis_std)

lx = 256
ly = 256

domain_decompose_1 = 2
domain_decompose_2 = 2

num_moments = 256
emin, emax = -3.02 * np.abs(t) - dis_std * sqrt(6), 3.02 * np.abs(t) + dis_std * sqrt(6)
configuration = kite.Configuration(divisions=[domain_decompose_1, domain_decompose_2], length=[lx, ly],
                                   boundaries=[True, True], is_complex=True, precision=1, spectrum_range=[emin, emax])

calculation = kite.Calculation(configuration)
direction = 'xy'
num_random = 1
calculation.dos(num_points=1000, num_moments=num_moments, num_random=num_random, num_disorder=1)
calculation.conductivity_dc(direction=direction, num_points=1000, num_moments=num_moments, num_random=num_random, num_disorder=1)

name = 'cond_{}_magn_{:.2f}T_dis_std_{:.1f}_moments_{}_random_{}_lx_{}_ly_{}.h5'
# configure the *.h5 file
full_name = name.format(direction, bfield, dis_std, num_moments, num_random, lx, ly)
kite.config_system(lat, configuration, calculation, modification=modification, filename=full_name, disorder=disorder)

In [ ]:
pb.utils.tic()  # measure the time
kite.KITEx(full_name)
pb.utils.toc('Time for moments calculation ')

pb.utils.tic()  # measure the time
kite.KITEtools(full_name + ' --CondDC -T 50 -F -0.5 0.5 10000 -E 4000 -K green 0.01 -N condDC_gr_magn_{}.dat'.format(direction))
kite.KITEtools(full_name + ' --DOS -F -0.5 0.5 100 -K green 0.01 -N dos_gr_magn.dat')
pb.utils.toc('Time for reconstructing ')

In [ ]:
from pybinding.constants import hbar
from pybinding.repository import graphene
from math import sqrt

def landau_level(magnetic_field: float, n: int):
    """ Calculate the energy of Landau level n in the given magnetic field. """
    lb = sqrt(hbar / magnetic_field)
    return hbar * (graphene.vf * 10 ** -9) / lb * sqrt(2 * n)


cond = np.loadtxt('condDC_gr_magn_{}.dat'.format(direction))
dos_raw = np.loadtxt('dos_gr_magn.dat')

plt.plot(cond[:,0], cond[:,1], c='C0')

ax1 = plt.gca()
y_max = 8

for i in np.arange(-5, 5, 1):
    ell = landau_level(magnetic_field=bfield, n=np.abs(i))
    ax1.plot((ell, ell), (-y_max, y_max), '--k', alpha=0.1)
    ax1.plot((-ell, -ell), (-y_max, y_max), '--k', alpha=0.1)

plt.yticks(np.linspace(-y_max, y_max, 9))
plt.xlim([-0.5,0.5])
plt.ylim([-y_max,y_max])
ax1.set_ylabel(r'$\mathrm{\sigma_{xx, xy}~({\rm e^2/h}})$')
ax1.set_xlabel(r'$E~({\rm eV})$')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(dos_raw[:,0], dos_raw[:,1], c='C1')
ax2.set_ylim([0, 0.02])
plt.sca(ax2)
ax2.tick_params(axis='y', labelcolor='C1')
ax2.set_ylabel(r'$\mathrm{DOS~(a.u.)}$')